In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test_data.xlsx to test_data.xlsx
Saving train_data.xlsx to train_data.xlsx


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

In [ ]:
class LoanDefaultPrediction:
    def __init__(self, model_type='logistic_regression'):
        self.model_type = model_type
        self.model = None

    def load(self, data):
        """
        Load the dataset and separate features and target variable
        """
        self.df = data
        self.X = self.df.drop('loan_status', axis=1)
        self.y = self.df['loan_status']

    def preprocess(self):
        """
        Apply preprocessing steps like scaling numeric features and encoding categorical features
        """
        # Handle datetime columns, if any
        date_columns = self.X.select_dtypes(include=['datetime64']).columns
        for col in date_columns:
            self.X[col] = pd.to_datetime(self.X[col])
            self.X[f'{col}_year'] = self.X[col].dt.year
            self.X[f'{col}_month'] = self.X[col].dt.month
            self.X[f'{col}_day'] = self.X[col].dt.day
            self.X[f'{col}_weekday'] = self.X[col].dt.weekday
            self.X = self.X.drop([col], axis=1)

        # Encode categorical columns (example with 'home_ownership' and 'purpose')
        if 'home_ownership' in self.X.columns:
            self.X['home_ownership'] = self.X['home_ownership'].map({'OWN': 0, 'MORTGAGE': 1, 'RENT': 2})

        if 'purpose' in self.X.columns:
            self.X['purpose'] = self.X['purpose'].map({'debt_consolidation': 0, 'credit_card': 1, 'other': 2, 'home_improvement': 3})

        # Ensure there are no missing values - ONLY for numeric columns
        numeric_columns = self.X.select_dtypes(include=['number']).columns
        self.X[numeric_columns] = self.X[numeric_columns].fillna(self.X[numeric_columns].median())  # Or self.X.dropna() if you want to drop missing rows

        # Apply StandardScaler to numerical features
        scaler = StandardScaler()
        numeric_columns = ['annual_inc', 'loan_amnt', 'cibil_score']
        self.X[numeric_columns] = scaler.fit_transform(self.X[numeric_columns])

        # Check final data types
        print(self.X.dtypes)

    def train(self):
        """
        Train the model using a split of training and testing data
        """
        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)

        # Choose the model
        if self.model_type == 'logistic_regression':
            self.model = LogisticRegression(max_iter=1000)
        elif self.model_type == 'random_forest':
            self.model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

        # Train the model
        self.model.fit(X_train, y_train)

    def test(self):
        """
        Test the model and print evaluation metrics
        """
        y_pred = self.model.predict(self.X)
        print("Accuracy:", accuracy_score(self.y, y_pred))
        print("Classification Report:\n", classification_report(self.y, y_pred))

    def predict(self, X):
        """
        Predict loan default for new data
        """
        return self.model.predict(X)